In [3]:
import json
from rank_bm25 import BM25Okapi
from tqdm import tqdm

In [7]:
with open('data\corpus_evidence_unified.json', 'r') as f:
    evidence_corpus = json.load(f)

# evidence_corpus[0]

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\kuipe\AppData\Local\Temp\ipykernel_12024\952399766.py:1: SyntaxWarning: invalid escape sequence '\c'
  with open('data\corpus_evidence_unified.json', 'r') as f:


In [15]:
# Load the claims
with open(r'data\train_claims_quantemp.json', 'r') as f:
    claims = json.load(f)


In [18]:
corpus = [evidence for evidence in tqdm(evidence_corpus.values())]
tokenized_corpus = [doc.split() for doc in corpus]

100%|██████████| 423320/423320 [00:00<00:00, 3066773.01it/s]


In [20]:
bm25 = BM25Okapi(tokenized_corpus)

In [29]:
def retrieve_evidence(claim, k=100):
    tokenized_claim = claim.split()
    scores = bm25.get_scores(tokenized_claim)
    top_k_indices = scores.argsort()[-k:][::-1]
    top_k_documents = [corpus[idx] for idx in top_k_indices]
    return top_k_documents

In [30]:
# Prepare data for NLI model
def prepare_nli_input(claim, evidence):
    nli_input = {
        'claim': claim,
        'evidence': " ".join(evidence)  # Concatenate top-k evidence into a single string
    }
    return nli_input

In [34]:
# claims_subset = claims[:20]
nli_data = []
for claim in tqdm(claims):
    claim_text = claim['claim']
    evidence = retrieve_evidence(claim_text)
    nli_input = prepare_nli_input(claim_text, evidence)
    nli_data.append(nli_input)

  0%|          | 0/9935 [00:00<?, ?it/s]

  1%|          | 55/9935 [03:04<9:11:40,  3.35s/it] 


KeyboardInterrupt: 

In [33]:
print(len(nli_data[0]['evidence']))

19778


In [28]:
print(nli_data[0]['claim'])

In her budget speech, Nirmala Sitharaman claimed that the Government distributed 35,000 crore LED bulbs in the country.
